# O output desse código deve ser um arquivo com a extração das imagens de satélite com base nas coordenadas dos arquivos gerados anteriormente. O arquivo deve ser no formato ano-mes-dia

## 1. Importar bibliotecas

In [ ]:
import subprocess
import xarray as xr
import pandas as pd

## 2. Criar subpasta e funções lambdas para tratarem a nomeclatura

In [ ]:
mes2 = lambda x: '0'+str(x) if x < 10 else str(x)
dia2 = lambda x: '0'+str(x) if x < 10 else str(x)
subprocess.run(['mkdir','/content/drive/MyDrive/dados_finais/'])

CompletedProcess(args=['mkdir', '/content/drive/MyDrive/dados_finais/'], returncode=1)

## 3. Função para baixar imagens pelo site da nasa

In [ ]:
def download_dia(ano,mes,dia):
  mes_2 = mes2(mes)
  dia_2 = dia2(dia)
  nome = str(ano)+mes_2+dia_2
  try:
    link="https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/"+nome+"090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc"
    subprocess.run(["wget",'-c','-O'+nome,'--user=xxxxx','--password=xxxxxxx',link]) 
  except:
    pass

## 4. Função para ler o arquivo ano - mes gerado anteriormente

In [ ]:
def arq_ano_mes(ano,mes):
  df = pd.read_parquet('/content/drive/MyDrive/dados_transformados/'+str(ano)+'-'+str(mes))
  return df

## 5. Função para abrir a imagem baixada

In [ ]:
def arq_img(ano,mes,dia):
  try:
    temp = xr.open_dataset('/content/'+str(ano)+str(mes2(mes))+str(dia2(dia)))
    return temp
  except:
    temp = xr.open_dataset(None)
    return temp

## 6. Função que cria um dataFrame com o dia da imagem baixada e seta uma coluna temperatura com valor fixo que irá ser substituido posteriormente pelos pontos das imagens extraidas

In [ ]:
def df_leitura_dia(df):
  df2 = df.loc[df['DIA']==dia].reset_index()
  df2 = df2.assign(TEMPERATURA=1.00)
  return df2

## 7. Função que extrai os pontos das imagens

In [ ]:
def gerarDfTepm(df2):
  df2 = df2
  longitude = df2[['LONGITUDE']]
  latitude = df2[['LATITUDE']]
  tamanho = len(df2)
  posix = 0
  while posix < tamanho:
    gerarDf(df2,longitude,latitude,posix)
    posix = posix+1
  return df2

## 8. Função que extrai os pontos das imagens, ela que tem o trabalho de extrair de fato os pontos, enquanto a anterior cria um dataframe posicionando as imagens

In [ ]:
def gerarDf(df2,longitude,latitude,posix):
  df2.loc[posix,'TEMPERATURA']=temp.sel(lon=[longitude.LONGITUDE[posix]], lat=[latitude.LATITUDE[posix]]).analysed_sst.data[0][0][0]

## 9. Essa função tem o trabalho de remover a imagem após os pontos dela ser extraido, isso faz sentido porque as imagens são grandes

In [ ]:
def remove_arq(ano,mes,dia):
  mes_2 = mes2(mes)
  dia_2 = dia2(dia)
  nome = str(ano)+mes_2+dia_2
  subprocess.run(['rm','/content/'+nome])

## 10. Aqui está a junção de todo o processo e gerando um df

In [ ]:
ano = input('Ano: ')
mes = 1
dia = 1
while mes <= 12:
  df = arq_ano_mes(ano,mes)
  resultado = pd.DataFrame(data=None) 
  while dia <= 31:
    download_dia(ano,mes,dia)
    try:
      temp = arq_img(ano,mes,dia)
      df2 = df_leitura_dia(df)
      df2 = gerarDfTepm(df2)
      df2.to_parquet('/content/drive/MyDrive/dados_finais/'+str(ano)+'-'+str(mes)+'-'+str(dia))
      remove_arq(ano,mes,dia)
      dia = dia + 1
    except:
      remove_arq(ano,mes,dia)
      dia = dia + 1
  dia = 1
  mes = mes+1 

Ano: 2020
